# Solving a 2D Poisson problem by aggFEM

Domain:
-------
The domain is $[-1,1]^2$ while the interface is a ringe described by a
level set function. In the discretisation the level set function is
approximated with a piecewise linear interpolation. This approximate
geometry is then mapped by applying a mesh deformation resulting in
a higher order geometry approximation.

PDE problem:
------------
$$ 
\begin{equation}
\begin{aligned}
    - \Delta u &= f \quad &&\text{in}  \quad \Omega  \\
            u &= g_D \quad &&\text{on} \quad \partial \Omega 
\end{aligned}
\end{equation}
$$

The r.h.s. term f is chosen according to a manufactured solution.

Discretisation:
---------------
* Background finite element space restricted to active domain (CutFEM)

* Nitsche formulation to impose boundary conditions, see. e.g. [1]

* Element aggregation to deal with bad cuts.

Implementational aspects:
-------------------------
* Geometry approximation using isoparametric unfitted FEM

* A (sparse) direct solver is applied to solve the arising linear systems.

Literature:
-----------
- E. Burman, P. Hansbo, Fictitious domain finite element methods using
    cut elements: II. A stabilized Nitsche method, Appl. Num. Math.
    62(4):328-341, 2012.
    
- S. Badia, F. Verdugo, and A. F. Martín. The aggregated unfitted finite 
    element method for elliptic problems. Comput. Methods Appl. Mech. 
    Engrg., 336:533–553, 2018.  


***This code is from the demos of ngsxfem.***

In [1]:
# ------------------------------ LOAD LIBRARIES -------------------------------
from netgen.geom2d import SplineGeometry
from ngsolve import *
from ngsolve.internal import *
from xfem import *
from xfem.lsetcurv import *

ngsglobals.msg_level = 2

importing ngsxfem-2.1.2504


In [2]:
# -------------------------------- PARAMETERS ---------------------------------
# Quadrilateral (or simplicial mesh)
quad_mesh = False
# Mesh diameter
maxh = 0.1 / 2
# Finite element space order
order = 2
# Stabilization parameter for Nitsche
lambda_nitsche = 10 * order * order


In [3]:
# Geometry and Mesh
square = SplineGeometry()
square.AddRectangle((-1, -1), (1, 1), bc=1)
ngmesh = square.GenerateMesh(maxh=maxh, quad_dominated=quad_mesh)
mesh = Mesh(ngmesh)

 Generate Mesh from spline geometry


In [11]:
# Manufactured exact solution for monitoring the error
r2 = 3 / 4  # outer radius
r1 = 1 / 4  # inner radius
rc = (r1 + r2) / 2.0
rr = (r2 - r1) / 2.0
r = sqrt(x**2 + y**2)
levelset = IfPos(r - rc, r - rc - rr, rc - r - rr)

exact = (20 * (r2 - sqrt(x**2 + y**2)) * (sqrt(x**2 + y**2) - r1)).Compile()
coeff_f = - (exact.Diff(x).Diff(x) + exact.Diff(y).Diff(y)).Compile()
gD = exact

In [8]:
# Higher order level set approximation
lsetmeshadap = LevelSetMeshAdaptation(mesh, order=order, threshold=0.1,
                                      discontinuous_qn=True)
deformation = lsetmeshadap.CalcDeformation(levelset)
lsetp1 = lsetmeshadap.lset_p1
DrawDC(lsetp1,-1,1,mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [9]:
# Element, facet and dof marking w.r.t. boundary approximation with lsetp1:
ci = CutInfo(mesh, lsetp1)
els_hasneg = ci.GetElementsOfType(HASNEG) # active mesh
els_neg = ci.GetElementsOfType(NEG)
els_if = ci.GetElementsOfType(IF)


In [7]:
# Set up FE-Space
Vhbase = H1(mesh, order=order, dirichlet=[], dgjumps=False)
Vh = Restrict(Vhbase, els_hasneg)

gfu = GridFunction(Vh)

u, v = Vh.TrialFunction(), Vh.TestFunction()
h = specialcf.mesh_size
n = Normalize(grad(lsetp1))

In [10]:
print(help(dCut))

Help on function dCut in module xfem:

dCut(
    levelset,
    domain_type,
    order=None,
    subdivlvl=None,
    time_order=-1,
    levelset_domain=None,
    **kwargs
)
    Differential symbol for cut integration.

    Parameters
    ----------
    levelset : ngsolve.GridFunction
        The level set fct. describing the geometry
        (desirable: P1 approximation).
    domain_type : {POS, IF, NEG, mlset.DomainTypeArray}
        The domain type of interest.
    order : int
        Modify the order of the integration rule used.
    subdivlvl : int
        Number of additional subdivision used on cut elements to
        generate the cut quadrature rule. Note: subdivlvl >0 only
        makes sense if you don't provide a P1 level set function
        and no isoparametric mapping is used.
    definedon : Region
        Domain description on where the integrator is defined.
    vb : {VOL, BND, BBND}
        Integration on mesh volume or its (B)boundary. Default: VOL
        (if combined

In [12]:
# integration domains:
dx = dCut(lsetp1, NEG, definedonelements=els_hasneg, deformation=deformation)
ds = dCut(lsetp1, IF, definedonelements=els_if, deformation=deformation)

a = BilinearForm(Vh, symmetric=True)
# Diffusion term
a += grad(u) * grad(v) * dx
# Nitsche term
a += -grad(u) * n * v * ds
a += -grad(v) * n * u * ds
a += (lambda_nitsche / h) * u * v * ds

# R.h.s. term:
f = LinearForm(Vh)
f += coeff_f * v * dx - grad(v)*n*gD*ds + (lambda_nitsche / h) * gD * v * ds

# Assemble unstable system
a.Assemble()
f.Assemble()

Our original linear system is 
$$ Au=f. $$
By extrapolating the bad dofs into root (interior) cells, u is obtained through 
$$u=Pu_p,$$ 
where $u_p$ is the vector with respect to root dofs.
Then the original linear system becomes
$$ APu_p=f. $$
To maintain symmetry, we use 
$$ P^TAPu_p=p^Tf. $$
Denote $A_{emb}:=P^TAP$, we have $A_{emb}u_p = p^Tf.$

In [13]:
# Compute element patches for element aggregation
EA = ElementAggregation(mesh, els_neg, els_if)

# Set up embedding matrix for element aggregation
P = AggEmbedding(EA, Vh, deformation=deformation)

print(f'P shape: {P.shape}')
PT = P.CreateTranspose()

Aemb = PT @ a.mat @ P



P shape: (3343, 2780)


In [16]:
# Solve linear system
gfuE = Aemb.Inverse(inverse='sparsecholesky') * (PT * f.vec)
gfu.vec.data = P * gfuE

# Measure the error
l2error = sqrt(Integrate((gfu - exact)**2 * dx, mesh))
print("L2 Error: {0}".format(l2error))

# visualization:
Draw(deformation, mesh, "deformation")
DrawDC(lsetp1, gfu, 0, mesh, "uh", deformation=deformation)

L2 Error: 0.0002021812570303219


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [19]:
gfe = GridFunction(Vh)
gfe.Set(exact)
# Draw(exact,mesh)
DrawDC(lsetp1, gfe, 0, mesh, "uh", deformation=deformation)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [21]:
kappaminus = CutRatioGF(ci)
kappaminus_values = kappaminus.vec.FV().NumPy()
positive_values = [v for v in kappaminus_values if v > 0]
if positive_values:
    min_value_pythonic = min(positive_values)
    print(f"The smallest cut ratio is: {min_value_pythonic}")
else:
    print("There are no cut elements.")

The smallest cut ratio is: 3.2785464771385517e-06


In [27]:
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import svds

# A 可以是稀疏矩阵，也可以是普通 ndarray（会自动按稀疏方式处理）
# 例如 A = scipy.sparse.random(10000, 10000, density=0.0001)

# 计算最大奇异值
rows,cols,vals = a.mat.COO()
A = sp.csr_matrix((vals,(rows,cols)))
condition_number = np.linalg.cond(A.todense())
# _, s_max,_ = svds(A, k=1, which='LM')   # largest magnitude

# # 计算最小奇异值
# _, s_min,_ = svds(A, k=1, which='SM')   # smallest magnitude

# condition_number = s_max[0] / s_min[0]
print(f"{condition_number:.2e}")


7.79e+14


In [33]:
# print(help(Aemb))

In [40]:
rows,cols,vals = Aemb.COO()
Agg = sp.csr_matrix((vals,(rows,cols)))
condition_number_agg = np.linalg.cond(Agg.todense())
print(f"{condition_number_agg:.2e}")

7.24e+03
